# We will simulate the longitudinal analysis ramdom effects mixed linear model result.    
    Y = [Rent burdened, Rent overburdened, Median rent, Median house price]
    X = [Percent Airbnb (all), Percent Airbnb (active)]

In [1]:
import numpy as np
#%matplotlib notebook
import itertools as it
import matplotlib.pyplot as plt
from math import e, log
import operator

In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [50]:
Y_names = ['Rent burdened', 'Rent overburdened', 'Median rent', 'Median house price']

Y_percentiles = [[0, 2.7707, 3.0070, 3.1769, 3.3894, 3.5069, 3.8921],
                 [0, 2.7707, 3.0070, 3.1769, 3.3894, 3.5069, 3.8921],
                 [0, 2.4076, 2.6963, 2.9790, 3.3421, 3.5429, 3.9899],
                 [0, 2.4076, 2.6963, 2.9790, 3.3421, 3.5429, 3.9899],
                 [5.279152, 6.8418, 7.2174, 7.4125, 7.5154, 7.6203, 7.8275],
                 [5.279152, 6.8418, 7.2174, 7.4125, 7.5154, 7.6203, 7.8275],
                 [9.7742, 13.0521, 13.3160, 13.5652, 13.7462, 13.8685, 14.1110],
                 [9.7742, 13.0521, 13.3160, 13.5652, 13.7462, 13.8685, 14.1110]] # these are for log values
                                                                                 # (column) x (percentile)

X_names = ['Log percent Airbnb all', 'Log percent Airbnb active']
X_percentiles_avg = [[0, 0.5171, 0.6648, 0.8195, 0.9449, 1.1748, 1.6816],
                     [0, 0.5612, 0.69244, 0.8706, 1.0173, 1.2994, 1.8952]]   # these are for log, avg. over time
                                                                             # (all, active) x (percentile)

percentiles = [0, 10, 25, 50, 75, 90, 100]
time        = [1, 2, 3, 4]
year        = {1: '2013', 2:'2014', 3:'2015', 4:'2016'}

X_percentiles_t = [{1: [0, 0, 0, 0, 0, 0.002, 0.2341],
                    2: [0, 0, 0, 0, 0, 0.658802, 1.54087],
                    3: [0, 0, 0, 0, 0.317699, 1.15403, 1.93610],
                    4: [0, 2.0574, 2.5335, 2.9610, 3.4153, 3.7544, 4.6051]},
                   {1: [0, 0, 0, 0, 0, 0, 0.1234],
                    2: [0, 0, 0, 0, 0, 0.7468, 1.8912],
                    3: [0, 0, 0, 0, 0.42745, 1.25708, 2.20995],
                    4: [0, 2.2149, 2.6690, 3.16625, 3.5764, 3.91855, 4.6051]}] # for log values
                                                                           # (all,active)x(time)x(percentile)
X_percentiles_t = [{1: [np.nan, np.nan, np.nan, np.nan, 0, 0.002, 0.2341],
                    2: [np.nan, np.nan, np.nan, np.nan, 0, 0.658802, 1.54087],
                    3: [np.nan, np.nan, np.nan, 0, 0.317699, 1.15403, 1.93610],
                    4: [np.nan, 2.0574, 2.5335, 2.9610, 3.4153, 3.7544, 4.6051]},
                   {1: [np.nan, np.nan, np.nan, np.nan, np.nan, 0, 0.1234],
                    2: [np.nan, np.nan, np.nan, np.nan, 0, 0.7468, 1.8912],
                    3: [np.nan, np.nan, np.nan, 0, 0.42745, 1.25708, 2.20995],
                    4: [np.nan, 2.2149, 2.6690, 3.16625, 3.5764, 3.91855, 4.6051]}] # for log values
                                                                           # (all,active)x(time)x(percentile)

YX_names = list(it.product(Y_names, X_names))  # (columns)

print(YX_names)

cA = [0.196, 0.193, 0.003, 0.001, 0.030, 0.029, 0.172, 0.037] # column

cT = [0.020, 0.013, -0.017, -0.014, -0.030, -0.031, -0.248, -0.138] # column

[('Rent burdened', 'Log percent Airbnb all'), ('Rent burdened', 'Log percent Airbnb active'), ('Rent overburdened', 'Log percent Airbnb all'), ('Rent overburdened', 'Log percent Airbnb active'), ('Median rent', 'Log percent Airbnb all'), ('Median rent', 'Log percent Airbnb active'), ('Median house price', 'Log percent Airbnb all'), ('Median house price', 'Log percent Airbnb active')]


In [54]:
def y(column, t, per):
    term1 = cA[column]*X_percentiles_t[column%2][t][per]
    term2 = cT[column]*(t-2.5)
    term3 = Y_percentiles[column][per]
    term4 = -cA[column]*X_percentiles_avg[column%2][per]
    return e**(term1 + term2 + term3 + term4)


texts = [r'min',
         r'$10^{th}$ p.',
         r'$25^{th}$ p.',
         r'median',
         r'$75^{th}$ p.',
         r'$90^{th}$ p.',
         r'max']

for column in range(8):
    fig = plt.figure(figsize=(7,5))
    ax  = fig.add_subplot(111)
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    
    for t in range(1,5):
        xs  = [i/log(10) for i in X_percentiles_t[column%2][t]]
        ys  = [y(column, t, per) for per in range(7)]
        plt.plot(xs, ys, 'o-',  label = year[t])
    
        text_pos = [(xs[0] + 0.05, ys[0] + 0),
                    (xs[1] + 0.05, ys[1] - 0),
                    (xs[2] + 0.05, ys[2] - 0),
                    (xs[3] + 0.05, ys[3] - 0),
                    (xs[4] + 0.05, ys[4] + 0),
                    (xs[5] + 0.05, ys[5] + 0),
                    (xs[6] + 0.05, ys[6] + 0)]
        [ax.annotate(texts[per], xy = text_pos[per]) for per in range(7)]
        ax.legend(loc='lower right')
        
    plt.ylabel(YX_names[column][0])
    plt.xlabel(YX_names[column][1])
    plt.grid(True)
    plt.title('Simulation of {} vs. {} (2013-16)'.format(*YX_names[column]))
    plt.savefig('{}_vs_{}.png'.format(*YX_names[column]), dpi=300)

/home/vaibhav/.anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/home/vaibhav/.anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


ValueError: cannot convert float NaN to integer

<Figure size 504x360 with 1 Axes>

ValueError: cannot convert float NaN to integer

<Figure size 504x360 with 1 Axes>

...

<Figure size 504x360 with 1 Axes>

ValueError: cannot convert float NaN to integer

<Figure size 504x360 with 1 Axes>

ValueError: cannot convert float NaN to integer

<Figure size 504x360 with 1 Axes>